In [3]:
import os
import numpy as np
import subprocess as sp
import pickle as pkl
import pandas as pd
import shutil

In [4]:
mod_dir = r'C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys'

In [5]:
hgs_mod_dir = os.path.join(mod_dir,'hgs')
model_name = 'lys'
grok_file_path = os.path.join(hgs_mod_dir,model_name + '_e.grok')

In [6]:
grok_file_stem = model_name + '_e'

In [7]:
def CreateCoupledModelDir(mod_dir):
    # Create directories if they don't exist
    coupled_mod_dir = os.path.join(mod_dir,'coupled')
    try:
        os.mkdir(coupled_mod_dir)
    except:
        print(coupled_mod_dir + ' already exists')
    # Create hgs subdirectory
    coupled_mod_hgs_dir = os.path.join(coupled_mod_dir,'hgs')
    try:
        os.mkdir(coupled_mod_hgs_dir)
    except:
        print(coupled_mod_hgs_dir + ' already exists')
    # Create dssat subdirectory
    coupled_mod_dssat_dir = os.path.join(coupled_mod_dir,'dssat')
    try:
        os.mkdir(coupled_mod_dssat_dir)
    except:
        print(coupled_mod_dssat_dir + ' already exists')
    return coupled_mod_dir,coupled_mod_hgs_dir,coupled_mod_dssat_dir

In [8]:
def GetSpinUpHeadsOutputFile(hgs_mod_dir,coupled_mod_hgs_dir,grok_file_stem):
    # Identify spin up grok name
    heads_file_stem = grok_file_stem.split('_')[0] + '_suo.head_pm'
    # Get highest number head_pm output file
    heads_file = [x for x in os.listdir(hgs_mod_dir) if heads_file_stem in x][-1]
    # Get path of that file
    spin_up_heads_file_path = os.path.join(hgs_mod_dir,heads_file)
    # Get path of coupled model hgs dir
    coupled_spin_up_heads_file_path = os.path.join(coupled_mod_hgs_dir,heads_file)
    # Copy file
    shutil.copy(spin_up_heads_file_path,coupled_spin_up_heads_file_path)


In [9]:
def GetHGSPropsFiles(hgs_mod_dir,coupled_mod_hgs_dir,grok_file_stem):
    # Get model name
    model_name = grok_file_stem.split('_')[0]
    # Get file names
    mprops_name = model_name + '.mprops'
    oprops_name = model_name + '.oprops'
    etprops_name = model_name + '.etprops'
    for file in [mprops_name,oprops_name,etprops_name]:
        # Paths and copy
        shutil.copy(os.path.join(hgs_mod_dir,file),os.path.join(coupled_mod_hgs_dir,file))

In [10]:
def GetStandaloneGrokLines(grok_file_path):
    # Read grok file
    with open(grok_file_path,'r') as file_in:
        lines = file_in.readlines()
    return lines

In [11]:

def GetStandaloneGrokPrecSeries(standalone_grok_lines):
    # Get start index
    start = standalone_grok_lines.index('!!--Begin Precipitation Time Series Section--\n')
    # Get end index
    end = standalone_grok_lines.index('!!--End Precipitation Time Series Section--\n')
    # Get P Block
    plines = standalone_grok_lines[start:end]
    # Get p series
    start = plines.index('    time value table\n')
    end = plines.index('    end\n')
    # Get time series lines
    tslines = plines[start+1:end]
    # Get Daily P Series
    p = [float(x.split(' ')[5].strip()) for x in tslines]
    # Get end day
    end_day = len(p)
    return p, end_day


In [12]:
def CreateDailyCoupledGrokFileDay0(lines,day,p):
    ## P Section
    # Get start index
    pstart = lines.index('!!--Begin Precipitation Time Series Section--\n')
    # Get end index
    pend = lines.index('!!--End Precipitation Time Series Section--\n')
    # Build P entry
    pentry = f'    time value table\n    0.0 {P[day]:.2f}\n    end\n'
    ## Output Section
    # Get start index
    ostart = lines.index('!!--Begin Output Times Section--\n')
    # Get end index
    oend = lines.index('!!--End Output Times Section--\n')
    new_lines = lines[:pstart+1]+[pentry]+lines[pend:ostart+1]+['1.0\nend\n']+lines[oend:]
    return new_lines

In [ ]:
def CreateDailyCoupledGrokFileDayN(lines,day,p,grok_file_stem):
    ## IC Section
    # Get start index
    icstart = lines.index('!!--Begin Initial Head Section--\n')
    # Get end index
    icend = lines.index('!!--End Initial Head Section--\n')
    # Build IC
    icentry = '! Set initial heads from day n-1\nchoose nodes all\n\ninitial head from output file\n{0}day{1}o.head_pm.0001\n\nclear chosen nodes\n'.format(grok_file_stem,day-1)
    ## Flux Nodal Section
    # Get start index
    fnstart = lines.index('!!--Begin Flux Nodal for DSSAT ET Section--\n')
    # Get end index
    fnend = lines.index('!!--End Flux Nodal for DSSAT ET Section--\n')
    # Build FN
    fnentry = '! Set flux nodal to force DSSAT ET\nboundary condition\n    type\n    flux nodal\n\n    node set\n    coupled_section\n\n    time file table\n    0.0 nflux.txt\n    0.00069444 none\n    end\nend\n'
    ## P Section
    # Get start index
    pstart = lines.index('!!--Begin Precipitation Time Series Section--\n')
    # Get end index
    pend = lines.index('!!--End Precipitation Time Series Section--\n')
    # Build P entry
    pentry = f'    time value table\n    0.0 {p[day]:.2f}\n    end\n'
    ## Solute Transport IC Section
    # Get start index
    sticstart = lines.index('!!--Begin Solute Transport Initial Concentration Section--')
    # Get end index
    sticend = lines.index('!!--End Solute Transport Initial Concentration Section--')
    # Build IC Entry
    sticentry = '! NH4 and NO3 boundary initial concentrations from DSSAT model in root zone and hgs in non-coupled zone\n\nchoose nodes all\n\ninitial concentration from file\niconc.txt\n\n'
    ## Output Section
    # Get start index
    ostart = lines.index('!!--Begin Output Times Section--\n')
    # Get end index
    oend = lines.index('!!--End Output Times Section--\n')
    new_lines = lines[:icstart+1]+[icentry]+lines[icend:fnstart+1]+[fnentry]+lines[fnend:pstart+1]+[pentry]+lines[pend:sticstart+1]+sticentry+lines[sticend:ostart+1]+['1.0\nend\n']+lines[oend:]
    return new_lines

In [14]:
def WriteCoupledGrokFile(new_lines,day,coupled_mod_hgs_dir,grok_file_stem):
    new_grok_name = grok_file_stem + 'day{}'.format(day) + '.grok'
    new_grok_path = os.path.join(coupled_mod_hgs_dir,new_grok_name)
    with open(new_grok_path,'w') as file:
        for entry in new_lines:
            file.write(entry)

In [15]:
## Build Coupled Model
# Create Directory Structure
cmod_dir,cmod_hgs_dir,cmod_dssat_dir = CreateCoupledModelDir(mod_dir)
# Copy over necessary hgs files
GetSpinUpHeadsOutputFile(hgs_mod_dir,cmod_hgs_dir,grok_file_stem)
GetHGSPropsFiles(hgs_mod_dir,cmod_hgs_dir,grok_file_stem)
# Get standalone model grok lines and Prec series
standalone_grok_lines = GetStandaloneGrokLines(grok_file_path)
P, End_Day = GetStandaloneGrokPrecSeries(standalone_grok_lines)
# Iterate through days to build daily hgs models
for day in np.arange(0,End_Day):
    # Day 0 model
    if day == 0:
        # Build text lines
        new_cgrok_lines = CreateDailyCoupledGrokFileDay0(standalone_grok_lines,day,P)
        # Write out
        WriteCoupledGrokFile(new_cgrok_lines,day,cmod_hgs_dir,grok_file_stem)
    # All other Day models
    else:
        # Build text lines
        new_cgrok_lines = CreateDailyCoupledGrokFileDayN(standalone_grok_lines,day,P,grok_file_stem)
        # Write out
        WriteCoupledGrokFile(new_cgrok_lines,day,cmod_hgs_dir,grok_file_stem)

C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys\coupled already exists
C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys\coupled\hgs already exists
C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys\coupled\dssat already exists


## Controller

In [16]:
def CreateControllerBatchFile(coupled_mod_dir,coupled_mod_hgs_dir):
    controller_path = os.path.join(coupled_mod_hgs_dir,'Controller.bat')
    with open(controller_path,'w') as file:
        file.write('cd {}\ngrok > out_g.txt\nphgs > out_h.txt\nhsplot\n'.format(coupled_mod_hgs_dir))
    out_path = os.path.join(coupled_mod_hgs_dir,'out.txt')
    with open(out_path,'w') as file:
        file.write('')


In [17]:
def RunCoupledModelHGSDaily(day,coupled_mod_hgs_dir,grok_file_stem):
    grok_name = grok_file_stem + 'day{}'.format(day)
    # First, update batch.pfx
    print('Updating batch.pfx')
    batch_pfx_path = os.path.join(coupled_mod_hgs_dir,'batch.pfx')
    with open(batch_pfx_path,'w') as file:
        file.write(grok_name)
    # Then run Controller
    print('Running Model')
    sp.run(['Controller.bat'])

In [18]:
mapping_pkl_path = r'C:\\Users\\southa0000\\Documents\\HGS-DSSAT\\HGS-DSSAT\\examples\\lys\\mapping\\lys_mapping.p'
rz_node_order_file_path = r'C:\\Users\\southa0000\\Documents\\HGS-DSSAT\\HGS-DSSAT\\examples\\lys\\hgs\\rz_node_order.txt'
full_node_order_file_path = r'C:\\Users\\southa0000\\Documents\\HGS-DSSAT\\HGS-DSSAT\\examples\\lys\\hgs\\full_node_order.txt'

In [19]:
def BuildETTimeValueTable(mapping_pkl_path,rz_node_order_file_path,coupled_mod_dssat_dir,coupled_mod_hgs_dir):
    # Load node list
    with open(rz_node_order_file_path,'r') as file:
        lines = file.readlines()
    nodes = [int(x.strip()) for x in lines]
    # Load mapping pickle
    with open(mapping_pkl_path,'rb') as file:
        map_dict = pkl.load(file)
    # Blank list for vals
    vals = []
    # Iterate through nodes
    for node in nodes:
        # Get DSSAT location information
        dssat_model,sheet,area_stat = map_dict[node]
        # Get surface ET and half of soil layer 1 for sheet 0
        if sheet == 0:
            # Identify path to DSSAT Surface ET file
            dssat_data_path = os.path.join(coupled_mod_dssat_dir,str(dssat_model) + '_SurfaceET.csv')
            # Grab val up from surface file
            valup = pd.read_csv(dssat_data_path)['EOAA'].values[0]
            # Identify path to DSSAT Soil ET File
            dssat_data_path = os.path.join(coupled_mod_dssat_dir,str(dssat_model) + '_SoilET.csv')
            # Grab val down from soil file layer 1
            valdn = pd.read_csv(dssat_data_path)['ES{}D'.format(sheet + 1)].values[0]
            # Set value to full surface ET + 1/2 of layer 1 ET
            val = (valup + (0.5*valdn)) * -1 * 24. * 60. / 1000. * (1./area_stat)
        # For bottom node sheet, just get half of last DSSAT layer
        elif sheet == 10:
            # Identify path to DSSAT Soil ET File
            dssat_data_path = os.path.join(coupled_mod_dssat_dir,str(dssat_model) + '_SoilET.csv')
            # Grab val down from soil file layer 1
            valup = pd.read_csv(dssat_data_path)['ES{}D'.format(sheet)].values[0]
            # Set value to 1/2 of layer 10 ET
            val = ((0.5*valup)) * -1 * 24. * 60. / 1000. * (1./area_stat)
        # For all other sheets, take half of layer above and half of layer below
        else:
            # Identify path to DSSAT Soil ET File
            dssat_data_path = os.path.join(coupled_mod_dssat_dir,str(dssat_model) + '_SoilET.csv')
            # Grab val down from soil file layer 1
            valup = pd.read_csv(dssat_data_path)['ES{}D'.format(sheet)].values[0]
            # Grab val down from soil file layer 1
            valdn = pd.read_csv(dssat_data_path)['ES{}D'.format(sheet + 1)].values[0]
            # Set value to 1/2 of layer n ET and 1/2 of layer n+1 ET
            val = ((0.5*valdn) + (0.5*valup)) * -1 * 24. * 60. / 1000. * (1./area_stat)
        # Convert DSSAT total mm to HGS total m3 and then multiply by minutes in a day to force all to be taken out in first minute of day
        vals.append(val)
    # Write out nflux.txt file
    nflux_path = os.path.join(coupled_mod_hgs_dir,'nflux.txt')
    with open(nflux_path,'w') as file:
        file.write(str(len(vals))+'\n')
        lines = []
        for val in vals:
            lines.append(str(val)+'\n')
        lines[-1] = lines[-1][:-1]
        for line in lines:
            file.write(line)
        

In [93]:
def BuildSoluteICFile(mapping_pkl_path,rz_node_order_file_path,full_node_order_file_path,coupled_mod_dssat_dir,coupled_mod_hgs_dir,grok_file_stem,day):
    # Load rz node list
    with open(rz_node_order_file_path,'r') as file:
        lines = file.readlines()
    rz_nodes = [int(x.strip()) for x in lines]
    # Load full node list
    with open(full_node_order_file_path,'r') as file:
        lines = file.readlines()
    full_nodes = [int(x.strip()) for x in lines]
    # Get list of non-coupled nodes
    hgs_only_nodes = [x for x in full_nodes if not (x in rz_nodes)]
    # Load mapping pickle
    with open(mapping_pkl_path,'rb') as file:
        map_dict = pkl.load(file)
    # Blank list for vals
    vals = []
    ## Load yesterdays HGS NH3 and NO4 values
    # Get pm file path
    pm_file = grok_file_stem.split('_')[0] + '_eday{}o.pm.dat'.format(day-1)
    pm_file_path = os.path.join(coupled_mod_hgs_dir,pm_file)
    # Load lines
    with open(pm_file_path,'r') as file:
        lines = file.readlines()
    # Get last entry in file for solution time = 1
    start = [i for i, line in enumerate(lines) if 'SOLUTIONTIME=1.00000000000000' in line][0]
    # Get indexes for starts and ends of no3 and nh4 sections
    st1lines = lines[start:]
    start_no3 = st1lines.index('# NO3\n')
    sub_lines = st1lines[start_no3:]
    inds = [i for i, line in enumerate(sub_lines) if '#' in line]
    no3lines = sub_lines[inds[0]+1:inds[1]]
    nh4lines = sub_lines[inds[1]+1:inds[2]]
    hgs_no3_vals = []
    for line in no3lines:
        for num in line.strip().split():
            hgs_no3_vals.append(float(num))
    hgs_nh4_vals = []
    for line in nh4lines:
        for num in line.strip().split():
            hgs_nh4_vals.append(float(num))
    ## Load dssat vals
    # Get list of possible dssat model names
    dssat_models = set([map_dict[x][0] for x in map_dict.keys()])
    # Store dssat vals in dictionary
    dssat_no3_vals = {}
    for mdl in dssat_models:
        dssat_output_path = os.path.join(coupled_mod_dssat_dir,str(mdl) + '_SoilNO3.csv')
        dssat_no3_vals[mdl] = pd.read_csv(dssat_output_path)
    dssat_nh4_vals = {}
    for mdl in dssat_models:
        dssat_output_path = os.path.join(coupled_mod_dssat_dir,str(mdl) + '_SoilNH4.csv')
        dssat_nh4_vals[mdl] = pd.read_csv(dssat_output_path)
    ## Iterate through nodes and store values from correct source in list
    no3_vals = []
    for node in hgs_only_nodes:
        # Get concentration from hgs_vals and append to list
        no3_vals.append(hgs_no3_vals[-1*node])
    for node in rz_nodes:
        # Identify model and sheet number
        mdl,sheet,a_stat = map_dict[node]
        # Get concentration from dssat_vals, convert and append to list
        if sheet == 0:
            no3_vals.append(dssat_no3_vals[mdl].loc[0,'NI{}D'.format(sheet+1)]*1000.)
        elif sheet == 10:
            no3_vals.append(dssat_no3_vals[mdl].loc[0,'NI{}'.format(sheet)]*1000.)
        elif sheet == 9:
            no3_vals.append(0.5*(dssat_no3_vals[mdl].loc[0,'NI{}D'.format(sheet)]*1000.)+0.5*(dssat_no3_vals[mdl].loc[0,'NI{}'.format(sheet+1)]*1000.))
        else:
            no3_vals.append(0.5*(dssat_no3_vals[mdl].loc[0,'NI{}D'.format(sheet)]*1000.)+0.5*(dssat_no3_vals[mdl].loc[0,'NI{}D'.format(sheet+1)]*1000.))
    nh4_vals = []
    for node in hgs_only_nodes:
        # Get concentration from hgs_vals and append to list
        nh4_vals.append(hgs_nh4_vals[-1*node])
    for node in rz_nodes:
        # Identify model and sheet number
        mdl,sheet,a_stat = map_dict[node]
        # Get concentration from dssat_vals, convert and append to list
        if sheet == 0:
            nh4_vals.append(dssat_nh4_vals[mdl].loc[0,'NH{}D'.format(sheet+1)]*1000.)
        elif sheet == 10:
            nh4_vals.append(dssat_nh4_vals[mdl].loc[0,'NH{}'.format(sheet)]*1000.)
        elif sheet == 9:
            nh4_vals.append(0.5*(dssat_nh4_vals[mdl].loc[0,'NH{}D'.format(sheet)]*1000.)+0.5*(dssat_nh4_vals[mdl].loc[0,'NH{}'.format(sheet+1)]*1000.))
        else:
            nh4_vals.append(0.5*(dssat_nh4_vals[mdl].loc[0,'NH{}D'.format(sheet)]*1000.)+0.5*(dssat_nh4_vals[mdl].loc[0,'NH{}D'.format(sheet+1)]*1000.))
    # Write out to file
    iconc_path = os.path.join(coupled_mod_hgs_dir,'iconc.txt')
    with open(iconc_path,'w') as file:
        lines = []
        for val in no3_vals:
            lines.append(str(val)+'\n')
        for val in nh4_vals:
            lines.append(str(val)+'\n')
        for line in lines:
            file.write(line)




In [94]:
day = 1
BuildSoluteICFile(mapping_pkl_path,rz_node_order_file_path,full_node_order_file_path,cmod_dssat_dir,cmod_hgs_dir,grok_file_stem,day)

In [ ]:
## Run Daily models
# First, create controller batch file
CreateControllerBatchFile(cmod_dir,cmod_hgs_dir)
# Change to coupled model directory
os.chdir(cmod_hgs_dir)
# Iterate through days to run daily hgs models
for day in np.arange(0,2):
    print('Entering day ' + str(day))
    if day > 0:
        print('Creating nflux file for DSSAT ET')
        BuildETTimeValueTable(mapping_pkl_path,rz_node_order_file_path,cmod_dssat_dir,cmod_hgs_dir)
        BuildSoluteICFile(mapping_pkl_path,rz_node_order_file_path,full_node_order_file_path,cmod_dssat_dir,cmod_hgs_dir,grok_file_stem,day)
    RunCoupledModelHGSDaily(day,cmod_hgs_dir,grok_file_stem)

Entering day 0
Updating batch.pfx
Running Model
Entering day 1
Creating nflux file for DSSAT ET
Updating batch.pfx
Running Model
